In [ ]:
import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import Utils.DiaSourceTools as DSTools
import lsst.afw.display.ds9 as ds9
from astropy.time import Time

In [ ]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_sim/"

In [70]:
directory = DATADIR+"/output"
butler = dafPersist.Butler(directory) 
dayDiff = butler.get("calexp", {"visit":850586,"filter":'r', "ccd":14})

In [ ]:
dates = butler.queryMetadata("dayDiff_class_diaSrc", format=["patch"], dataId={"date":"2005-07-07"})
print dates

In [ ]:
patches = ["8,4"]
dates = ["2005-07-07"]

multi_matches = None
for date in dates:
    date_catalog = None
    for patch in patches:
        if butler.datasetExists("dayDiff_class_diaSrc", {"patch":patch,"filter":'r', "tract":0, "date":date}) == True:
            catalog = butler.get("dayDiff_class_diaSrc", {"patch":patch,"filter":'r', "tract":0, "date":date})
            if date_catalog is None: 
                date_catalog = catalog.copy()
            else:
                date_catalog.extend(catalog)
                
    if multi_matches is None and len(date_catalog)>0:
        multi_matches = afwTable.MultiMatch(date_catalog[0].schema, {'date':float})
    
    t = Time(date)
    multi_matches.add(date_catalog, {'date':t.mjd})
                
results = multi_matches.finish(removeAmbiguous=False)  

In [ ]:
lcs = DSTools.get_light_curves_from_multimatch_results2(results)

In [ ]:
len(lcs)